In [8]:
# Модуль для получения данных из PubMed и получения списка расстояний между авторами.
from Bio import Entrez,Medline
import Levenshtein as lev
import time

# Класс - структура для хранения данных об авторе
class Author:
    def __init__(self,id,name,coauthors,affiliations,abstract,article_title):
        self.id=id
        self.name=name
        self.coauthors=coauthors
        self.affiliations=affiliations
        self.abstract=abstract
        self.article_title=article_title
        
# Класс для подсчета расстояния между авторами
# содержит список пар авторов и расстояний между ними
class Pair:
        def __init__(self,id1,id2,pair_names,pair_coauthors,pair_affiliations):
            self.id1=id1
            self.id2=id2
            self.pair_names=pair_names
            self.pair_coauthors=pair_coauthors
            self.pair_affiliations=pair_affiliations
            
class Author_distance:
               
    def __init__(self,list_of_authors,distance):
        # создаем список для хранения пар авторов
        self.list_of_pairs=[]
        for i in range(0,len(list_of_authors)):
            for j in range(0,len(list_of_authors)):
                if (i!=j):
                    id1 = list_of_authors[i].id
                    id2 = list_of_authors[j].id
                    pair_names =distance(list_of_authors[i].name,list_of_authors[j].name)
                    pair_coauthors =distance(list_of_authors[i].coauthors,list_of_authors[j].coauthors)
                    pair_affiliations =distance(list_of_authors[i].affiliations,list_of_authors[j].affiliations)
                    self.list_of_pairs.append(Pair(id1,id2,pair_names,pair_coauthors,pair_affiliations))
    
    

# Получение данных из PubMed
def obt_data(N, query):
    def search(query):
        Entrez.email='12113@gmail.com'
        handle=Entrez.esearch(db='pubmed',sort = 'revelance', retmax =N,term=query)
        record=Entrez.read(handle)
        idList=record["IdList"]
        handle=Entrez.efetch(db='pubmed', id = idList, retmode='xml')
        results=Entrez.read(handle)
        return results
    
    data=search(query)
    
    # Получение аффилиаций конкретного автора
    def affiliation(data,i,j):
        return '.'.join(data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"][j]['AffiliationInfo'][0]['Affiliation'].split(","))


    # Получение абстракта статьи
    def abstract(data,i):
            return data["PubmedArticle"][i]["MedlineCitation"]["Article"]['Abstract']['AbstractText']

    # Получение имени автора
    def name(data,i,j):
            return data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"][j]['LastName']+' '+data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"][j]['ForeName']+' ' +data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"][j]['Initials'] 

    # Получение заголовка статьи
    def article_title(data,i):
        return data["PubmedArticle"][i]["MedlineCitation"]["Article"]['ArticleTitle']

    # Получение соавторов  
    def coauthors(data,i,j):
        coauthors=[]
        author=name(data,i,j)
        for k in range(0,len(data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"])):
            coauthors.append(name(data,i,k))
        return ''.join(coauthors)

    # Получение количество авторов статьи
    def numberOfAuthors(data,i):
        return len(data["PubmedArticle"][i]["MedlineCitation"]["Article"]["AuthorList"])

    # Сбор всех данных в список авторов
    author_list=[]
    id=0
    for i in range(0,N):
        for j in range(0,numberOfAuthors(data,i)):
            try:
                author_list.append(Author(id,name(data,i,j),coauthors(data,i,j),affiliation(data,i,j),abstract(data,i),article_title(data,i)))
                id=id+1
            except IndexError:
                pass
            except KeyError:
                pass
    return author_list

In [ ]:
# Модуль для получения обучающей выборки
from ipywidgets import widgets

# Структура содеражщая пару id авторов и информацию являются ли они одним и тем же автором.
class Training_set:
        def __init__(self,id1,id2,same):
            self.id1=id1
            self.id2=id2
            self.same=same

# функция для простого отображения виджетов с информацией о данном авторе.
def print_info(id):
    author_text=widgets.Textarea(
    value=list_of_authors[id].name,
    disabled=True
    )
    coauthors_text=widgets.Textarea(
    value=list_of_authors[id].coauthors,
    disabled=True
    )
    affiliations_text=widgets.Textarea(
    value=list_of_authors[id].affiliations,
    disabled=True
    )
    return author_text,coauthors_text,affiliations_text

# функция закрытия всех переданных виджетов.
def close_widgets(name,coauthors,aff,box):
    name.close()
    coauthors.close()
    aff.close()
    box.close()
    
# функция для вывода графического интерфейса для получения обучающей выборки. Pair- набор пар, i- с какой пары начать.
def training(pair,i):
   
    # Формируем виджеты для каждого автора
    a=print_info(pair[i].id1)
    b=print_info(pair[i].id2)
    
    name=widgets.HBox([a[0],b[0]])
    coauthors=widgets.HBox([a[1],b[1]])
    aff=widgets.HBox([a[2],b[2]])
    
    # формируем виджеты кнопок
    yes_button= widgets.Button(description="YES")
    no_button= widgets.Button(description="NO")
    cancel_button= widgets.Button(description="Cancel")
    skip_button= widgets.Button(description="Skip")
    box = widgets.HBox([yes_button,no_button,skip_button,cancel_button])
    
    def on_yes_button_clicked(button):
        training_set.append(Training_set(pair[i].id1,pair[i].id2,1))
        close_widgets(name,coauthors,aff,box)
        training(pair,i+1)
    yes_button.on_click(on_yes_button_clicked)
    
    def on_no_button_clicked(button):
        training_set.append(Training_set(pair[i].id1,pair[i].id2,0))
        close_widgets(name,coauthors,aff,box)
        training(pair,i+1)
    no_button.on_click(on_no_button_clicked)

    def on_cancel_button_clicked(button):
        close_widgets(name,coauthors,aff,box)
        print("Вы остановились на паре номер: "+str(i))
    cancel_button.on_click(on_cancel_button_clicked)

    def on_skip_button_clicked(button):
        close_widgets(name,coauthors,aff,box)
        training(pair,i+1)
    skip_button.on_click(on_skip_button_clicked)
        
    display(name,coauthors,aff,box)
   

In [ ]:
list_of_authors=obt_data(20,"miRna")
list_of_pairs=Author_distance(list_of_authors,lev.distance)
training(list_of_pairs.list_of_pairs,0)